# MOVIE REVIEW SENTIMENT ANALYSIS WITH IMDB REVIEWS

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)

2.13.0


## Downloading the dataset

In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0LGUEM/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0LGUEM/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0LGUEM/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Splitting the dataset

In [3]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Loop over all training examples and save the sentences and labels
for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

# Loop over all training examples and save the sentences and labels
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

## Preprocessing

In [5]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [6]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post' # when a sequence is longer than maxlen, the extra elements at the end of the sequence will be removed or truncated to match the maxlen.
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

## Model Creation and Training
We'll look at 2 ways to create the model.
- Flatten()
- GlobalAveragePooling1D()

GlobalAveragePooling1D() averages across the vector to flatten it out. The model summary for this is a little simpler and it is a little faster.


### Method 1: Flatten()

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171533 (670.05 KB)
Trainable params: 171533 (670.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

num_epochs = 10
history1 = model.fit(padded, # is the x_train
                      training_labels_final, # is the y_train
                      epochs = num_epochs,
                      validation_data = (testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.4931 - accuracy: 0.7424 - val_loss: 0.3391 - val_accuracy: 0.8530
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2381 - accuracy: 0.9080 - val_loss: 0.3590 - val_accuracy: 0.8449
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0878 - accuracy: 0.9776 - val_loss: 0.4443 - val_accuracy: 0.8310
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0222 - accuracy: 0.9973 - val_loss: 0.5207 - val_accuracy: 0.8302
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0061 - accuracy: 0.9994 - val_loss: 0.5948 - val_accuracy: 0.8274
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.6346 - val_accuracy: 0.8310
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 8.5330e-04 - accuracy: 1.0000 - val_loss: 0.6770 - val_accuracy: 0.8320
Ep

Since accuracy is 1.0000, we're overfitting.

### Method 2: GlobalAveragePooling1D()

In [9]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_2 (Dense)             (None, 6)                 102       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160109 (625.43 KB)
Trainable params: 160109 (625.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

num_epochs = 10
history2 = model2.fit(padded, # is the x_train
                      training_labels_final, # is the y_train
                      epochs = num_epochs,
                      validation_data = (testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 7ms/step - loss: 0.5481 - accuracy: 0.7605 - val_loss: 0.3803 - val_accuracy: 0.8423
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3300 - accuracy: 0.8622 - val_loss: 0.3313 - val_accuracy: 0.8589
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2693 - accuracy: 0.8912 - val_loss: 0.3392 - val_accuracy: 0.8536
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2330 - accuracy: 0.9100 - val_loss: 0.3468 - val_accuracy: 0.8526
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2061 - accuracy: 0.9227 - val_loss: 0.3680 - val_accuracy: 0.8466
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.1854 - accuracy: 0.9326 - val_loss: 0.3946 - val_accuracy: 0.8422
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1676 - accuracy: 0.9409 - val_loss: 0.4280 - val_accuracy: 0.8363
Epoch 

Even though this method was faster, its accuracy was a little bit less than the other method. So, we'll carry on with the model which used `Flatten()`.

## Visualizing Word Embeddings

In [11]:
e = model.layers[0] # we'll get the results of the embeddings layes which is layer 0
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(10000, 16)


In [12]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

In [13]:
import io

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, vocab_size):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

# Close the files
out_v.close()
out_m.close()

When running this on Colab, you can run the code below to download the files. Otherwise, you can see the files in your current working directory and download it manually.

In [14]:
# Import files utilities in colab
try:
    from google.colab import files
except ImportError:
    pass

# Download the files
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now you can go to the [Tensorflow Embedding Projector](https://projector.tensorflow.org/) and load the two files you downloaded to see the visualization. You can search for words like `worst` and `fantastic` and see the other words closely located to these.